<a href="https://colab.research.google.com/github/davidbro-in/natural-language-processing/blob/main/3_custom_embedding_using_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://www.inove.com.ar"><img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center"></a>


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [ ]:
# Descargar la carpeta de dataset
import os
import gdown
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('simpsons_dataset.zip', os.F_OK) is False:
        url = 'https://drive.google.com/uc?id=1VLVgb3fD02XrF2V5OruKvQTOT91juXcs&export=download'
        output = 'songs_dataset.zip'
        gdown.download(url, output, quiet=False)
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

Downloading...
From: https://drive.google.com/uc?id=1VLVgb3fD02XrF2V5OruKvQTOT91juXcs&export=download
To: /content/songs_dataset.zip
100%|██████████| 2.09M/2.09M [00:00<00:00, 58.3MB/s]


In [ ]:
# Posibles bandas
os.listdir("./songs_dataset/")

['paul-simon.txt',
 'bjork.txt',
 'leonard-cohen.txt',
 'dolly-parton.txt',
 'britney-spears.txt',
 'bieber.txt',
 'cake.txt',
 'kanye.txt',
 'lil-wayne.txt',
 'Kanye_West.txt',
 'missy-elliott.txt',
 'johnny-cash.txt',
 'nickelback.txt',
 'amy-winehouse.txt',
 'michael-jackson.txt',
 'disney.txt',
 'rihanna.txt',
 'bob-marley.txt',
 'joni-mitchell.txt',
 'bruce-springsteen.txt',
 'lorde.txt',
 'bob-dylan.txt',
 'dj-khaled.txt',
 'nicki-minaj.txt',
 'bruno-mars.txt',
 'alicia-keys.txt',
 'notorious-big.txt',
 'lady-gaga.txt',
 'ludacris.txt',
 'janisjoplin.txt',
 'dickinson.txt',
 'notorious_big.txt',
 'patti-smith.txt',
 'prince.txt',
 'nirvana.txt',
 'jimi-hendrix.txt',
 'eminem.txt',
 'beatles.txt',
 'lin-manuel-miranda.txt',
 'adele.txt',
 'drake.txt',
 'kanye-west.txt',
 'dr-seuss.txt',
 'al-green.txt',
 'blink-182.txt',
 'nursery_rhymes.txt',
 'Lil_Wayne.txt',
 'radiohead.txt',
 'r-kelly.txt']

In [ ]:
!wget https://www.gutenberg.org/cache/epub/31193/pg31193.txt

--2021-09-16 22:02:02--  https://www.gutenberg.org/cache/epub/31193/pg31193.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114408 (112K) [text/plain]
Saving to: ‘pg31193.txt’

pg31193.txt         100%[===================>] 111.73K  --.-KB/s    in 0.1s    

2021-09-16 22:02:02 (1.11 MB/s) - ‘pg31193.txt’ saved [114408/114408]



In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('/content/pg31193.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,The Project Gutenberg EBook of Manifesto of th...
1,Karl Marx and Frederick Engels
2,This eBook is for the use of anyone anywhere a...
3,almost no restrictions whatsoever. You may co...
4,re-use it under the terms of the Project Guten...


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1796


### 1 - Preprocesamiento

In [ ]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:2]

[['the',
  'project',
  'gutenberg',
  'ebook',
  'of',
  'manifesto',
  'of',
  'the',
  'communist',
  'party',
  'by'],
 ['karl', 'marx', 'and', 'frederick', 'engels']]

### 2 - Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1796


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 523


### 3 - Entrenar el modelo generador

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 141628.34375
Loss after epoch 1: 74423.96875
Loss after epoch 2: 72339.6875
Loss after epoch 3: 72246.21875
Loss after epoch 4: 71785.125
Loss after epoch 5: 71975.09375
Loss after epoch 6: 72338.125
Loss after epoch 7: 73623.4375
Loss after epoch 8: 73091.625
Loss after epoch 9: 73558.5625
Loss after epoch 10: 73450.9375
Loss after epoch 11: 73388.0625
Loss after epoch 12: 72947.625
Loss after epoch 13: 67410.8125
Loss after epoch 14: 63964.0
Loss after epoch 15: 63015.125
Loss after epoch 16: 62892.125
Loss after epoch 17: 63648.5
Loss after epoch 18: 63255.5
Loss after epoch 19: 63602.625


(166408, 357940)

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["capital"], topn=10)

[('national', 0.9925433397293091),
 ('become', 0.9920361042022705),
 ('wage', 0.9912710785865784),
 ('english', 0.9899228811264038),
 ('developed', 0.9898823499679565),
 ('since', 0.9897341728210449),
 ('were', 0.9893654584884644),
 ('where', 0.9893307685852051),
 ('because', 0.9890304803848267),
 ('system', 0.9889686107635498)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(positive=["right"], topn=10)

[('little', 0.998960554599762),
 ('express', 0.998708963394165),
 ('help', 0.9986742734909058),
 ('less', 0.9984412789344788),
 ('written', 0.9984068870544434),
 ('say', 0.9983329772949219),
 ('given', 0.9983271956443787),
 ('money', 0.9983250498771667),
 ('either', 0.9982715249061584),
 ('except', 0.9982256889343262)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["power"], topn=10)

[('order', 0.9991225004196167),
 ('supremacy', 0.9990338087081909),
 ('character', 0.9988534450531006),
 ('whose', 0.9987103343009949),
 ('made', 0.9986703991889954),
 ('great', 0.9985951781272888),
 ('then', 0.9982390403747559),
 ('england', 0.9981627464294434),
 ('germany', 0.9981406927108765),
 ('life', 0.9981369972229004)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["state"], topn=5)

[('existing', 0.9963316917419434),
 ('proletariat', 0.9959862232208252),
 ('between', 0.9944427609443665),
 ('classes', 0.994326114654541),
 ('movement', 0.9935375452041626)]

In [ ]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["work"])

[('class', -0.3217197060585022),
 ('working', -0.36751803755760193),
 ('modern', -0.38811030983924866),
 ('has', -0.4405629634857178),
 ('conditions', -0.45894092321395874),
 ('against', -0.4654366970062256),
 ('production', -0.4761267304420471),
 ('bourgeois', -0.500950038433075),
 ('political', -0.5024511814117432),
 ('industry', -0.5061120390892029)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab
